In [41]:
import numpy as np 
import pandas as pd 
from datetime import date

In [42]:
ratios = [0,0.236, 0.382, 0.5 , 0.618, 0.786,1,1.414,1.618]
fiblevels = np.array(ratios)

In [43]:
def RSI(series, period=14):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    return 100 - 100 / (1 + rs)


# calculating Stoch RSI (gives the same values as TradingView)
# https://www.tradingview.com/wiki/Stochastic_RSI_(STOCH_RSI) 
def STRSIA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()
    return stochrsi, stochrsi_K, stochrsi_D

In [44]:
ratios

[0, 0.236, 0.382, 0.5, 0.618, 0.786, 1, 1.414, 1.618]

In [5]:
ada15 = pd.read_json('BN-main/ADA_USDT-1h.json')
ada15

,0,1,2,3,4,5
0,1546300800000,0.04057,0.04059,0.03995,0.04035,5902479.60
1,1546304400000,0.04040,0.04048,0.03987,0.04000,3739370.50
2,1546308000000,0.04004,0.04004,0.03964,0.03991,3615775.00
3,1546311600000,0.03988,0.04021,0.03985,0.04020,2419828.60
4,1546315200000,0.04020,0.04068,0.04017,0.04043,5649221.70
...,...,...,...,...,...,...
22152,1626235200000,1.21680,1.23060,1.21220,1.22100,7593028.91
22153,1626238800000,1.22080,1.22170,1.19800,1.20280,15978748.32
22154,1626242400000,1.20280,1.22000,1.19980,1.20740,7789568.81
22155,1626246000000,1.20750,1.22410,1.20660,1.22200,6168385.93


In [6]:
ada15.columns=['date','open','high','low','close','volume']
ada15

,date,open,high,low,close,volume
0,1546300800000,0.04057,0.04059,0.03995,0.04035,5902479.60
1,1546304400000,0.04040,0.04048,0.03987,0.04000,3739370.50
2,1546308000000,0.04004,0.04004,0.03964,0.03991,3615775.00
3,1546311600000,0.03988,0.04021,0.03985,0.04020,2419828.60
4,1546315200000,0.04020,0.04068,0.04017,0.04043,5649221.70
...,...,...,...,...,...,...
22152,1626235200000,1.21680,1.23060,1.21220,1.22100,7593028.91
22153,1626238800000,1.22080,1.22170,1.19800,1.20280,15978748.32
22154,1626242400000,1.20280,1.22000,1.19980,1.20740,7789568.81
22155,1626246000000,1.20750,1.22410,1.20660,1.22200,6168385.93


In [7]:
start = pd.to_datetime('2020-01-01')
ada15.date=pd.to_datetime(ada15['date'],unit='ms')
ada15


,date,open,high,low,close,volume
0,2019-01-01 00:00:00,0.04057,0.04059,0.03995,0.04035,5902479.60
1,2019-01-01 01:00:00,0.04040,0.04048,0.03987,0.04000,3739370.50
2,2019-01-01 02:00:00,0.04004,0.04004,0.03964,0.03991,3615775.00
3,2019-01-01 03:00:00,0.03988,0.04021,0.03985,0.04020,2419828.60
4,2019-01-01 04:00:00,0.04020,0.04068,0.04017,0.04043,5649221.70
...,...,...,...,...,...,...
22152,2021-07-14 04:00:00,1.21680,1.23060,1.21220,1.22100,7593028.91
22153,2021-07-14 05:00:00,1.22080,1.22170,1.19800,1.20280,15978748.32
22154,2021-07-14 06:00:00,1.20280,1.22000,1.19980,1.20740,7789568.81
22155,2021-07-14 07:00:00,1.20750,1.22410,1.20660,1.22200,6168385.93


In [45]:
import os

start = pd.to_datetime('2021-04-01')
end = pd.to_datetime('2021-05-19')

directory = '/home/haytham/crypt/freqtrade/user_data/data/binance'
table = []
for filename in os.listdir(directory):
    if filename.endswith('USDT-1h.json'):
        ada15 = pd.read_json(directory+'/'+filename)
        ada15.columns=['date','open','high','low','close','volume']
        ada15.date=pd.to_datetime(ada15['date'],unit='ms')
        ada2021 = ada15.loc[(ada15['date']> start)& (ada15['date']< end)].copy()
        print(filename)

        ada2021['RSI'] = RSI(ada2021['close'])
        ada2021['RSI4']=RSI(ada2021['close'],56)
        ada2021['RSI8']=RSI(ada2021['close'],112)
        ada2021['STRSI'] = STRSIA(ada2021['close'])[0]
        ada2021['STRSIK'] = STRSIA(ada2021['close'])[1]
        ada2021['STRSID'] = STRSIA(ada2021['close'])[2]

        ada2021['STRSI4'] = STRSIA(ada2021['close'],56,12,12)[0]
        ada2021['STRSIK4'] = STRSIA(ada2021['close'],56,12,12)[1]
        ada2021['STRSID4'] = STRSIA(ada2021['close'],56,12,12)[2]

        ada2021['STRSI8'] = STRSIA(ada2021['close'],112,24,24)[0]
        ada2021['STRSIK8'] = STRSIA(ada2021['close'],112,24,24)[1]
        ada2021['STRSID8'] = STRSIA(ada2021['close'],112,24,24)[2]



ada2021

DEGO_USDT-1h.json
YFIUP_USDT-1h.json
ONT_USDT-1h.json
XVS_USDT-1h.json
DOGE_USDT-1h.json
TROY_USDT-1h.json
EPS_USDT-1h.json
NEO_USDT-1h.json
XLMUP_USDT-1h.json
ASR_USDT-1h.json
EOSUP_USDT-1h.json
ZEN_USDT-1h.json
NKN_USDT-1h.json
BNBUP_USDT-1h.json
SUN_USDT-1h.json
TFUEL_USDT-1h.json
ARDR_USDT-1h.json
ACM_USDT-1h.json
BTG_USDT-1h.json
BTT_USDT-1h.json
INJ_USDT-1h.json
MTL_USDT-1h.json
XTZUP_USDT-1h.json
XLM_USDT-1h.json
DODO_USDT-1h.json
SRM_USDT-1h.json
YFII_USDT-1h.json
SUPER_USDT-1h.json
DGB_USDT-1h.json
COCOS_USDT-1h.json
XMR_USDT-1h.json
FTM_USDT-1h.json
FORTH_USDT-1h.json
ENJ_USDT-1h.json
HIVE_USDT-1h.json
NULS_USDT-1h.json
HOT_USDT-1h.json
TKO_USDT-1h.json
ETC_USDT-1h.json
UNI_USDT-1h.json
FET_USDT-1h.json
CELR_USDT-1h.json
MFT_USDT-1h.json
MATIC_USDT-1h.json
GTO_USDT-1h.json
FUN_USDT-1h.json
ETH_USDT-1h.json
LTO_USDT-1h.json
PNT_USDT-1h.json
LINKUP_USDT-1h.json
DNT_USDT-1h.json
ARPA_USDT-1h.json
PAX_USDT-1h.json
SLP_USDT-1h.json
AKRO_USDT-1h.json
KSM_USDT-1h.json
BTC_USDT-1h.js

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,STRSIK,STRSID,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8
0,2021-04-30 12:00:00,8.54,14.80,8.02,12.75,1158084.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-04-30 13:00:00,12.74,12.75,11.44,11.95,536910.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-30 14:00:00,11.91,12.10,11.16,11.22,396754.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-04-30 15:00:00,11.21,13.89,11.18,12.01,973431.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-04-30 16:00:00,12.01,12.32,11.36,11.70,341741.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,2021-05-18 19:00:00,11.33,11.66,11.28,11.65,80917.2,51.051283,48.564403,48.244333,0.281616,0.162763,0.205092,0.686306,0.718941,0.742021,0.660267,0.589638,0.434715
440,2021-05-18 20:00:00,11.63,11.73,11.40,11.45,55813.4,47.805910,47.882057,47.959729,0.109192,0.130269,0.162953,0.591742,0.701657,0.747213,0.573195,0.597034,0.445303
441,2021-05-18 21:00:00,11.43,11.48,11.32,11.45,57819.3,47.805910,47.882057,47.959729,0.109192,0.166666,0.153233,0.591742,0.683316,0.747934,0.573195,0.610577,0.456478
442,2021-05-18 22:00:00,11.45,11.64,11.38,11.44,29581.3,47.630327,47.847210,47.945331,0.099863,0.106082,0.134339,0.586913,0.648893,0.741346,0.568790,0.625261,0.468633


In [9]:
#maxlevels = [13,13,13,13,13,13,13]
#maxlevels = np.array(maxlevels)
#maxlevels

In [10]:
#minlevels = [11,11,11,11,11,11,11]
#minlevels = np.array(minlevels)
#minlevels

In [42]:
ada2021
under02=ada2021[(ada2021['STRSI']<0.2)]
under02a=under02[['RSI','STRSI','STRSI4','STRSI8']]
under02a


,RSI,STRSI,STRSI4,STRSI8
19695,52.496295,0.063070,NaN,NaN
19698,52.653332,0.006900,NaN,NaN
19699,49.501558,0.000000,NaN,NaN
19700,45.232229,0.000000,NaN,NaN
19701,41.103301,0.000000,NaN,NaN
...,...,...,...,...
20793,41.095251,0.000000,0.110664,0.070916
20794,39.865669,0.000000,0.090036,0.053170
20795,39.482971,0.000000,0.083768,0.047807
20801,41.925941,0.190634,0.127531,0.050880


In [12]:
high = ada2021.high
low = ada2021.low
maximums5 = high.rolling(5,5)
max5 = maximums5.max()
minimums5 = low.rolling(5,5)
min5 = minimums5.min()
maximums10 = high.rolling(10,10)
max10 = maximums10.max()
minimums10 = low.rolling(10,10)
min10 = minimums10.min()


In [13]:
len(max5)

431

In [14]:

ada2021['max5'] = max5
ada2021['max10'] = max10
ada2021['min5'] = min5
ada2021['min10'] = min10

ada2021.tail()

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8,max5,max10,min5,min10
20084,2021-04-18 19:00:00,1.24043,1.26295,1.22222,1.26061,20901192.7,48.748560,45.402254,47.743760,1.000000,...,0.514276,0.311380,0.177651,0.224025,0.090223,0.086258,1.26399,1.26399,1.20615,1.12500
20085,2021-04-18 20:00:00,1.26087,1.27659,1.25200,1.27659,14791770.3,51.589183,46.288811,48.237966,1.000000,...,0.571455,0.344837,0.198231,0.268770,0.097770,0.084299,1.27659,1.27659,1.20673,1.12500
20086,2021-04-18 21:00:00,1.27658,1.27700,1.24508,1.27398,15814949.9,51.091100,46.164152,48.162900,0.981588,...,0.563415,0.376721,0.221739,0.264368,0.104938,0.082862,1.27700,1.27700,1.22028,1.12932
20087,2021-04-18 22:00:00,1.27360,1.29635,1.27153,1.29427,18891820.3,54.748728,47.287794,48.788015,1.000000,...,0.635885,0.416118,0.248665,0.314807,0.114932,0.081968,1.29635,1.29635,1.22222,1.18360
20088,2021-04-18 23:00:00,1.29433,1.29527,1.26500,1.27693,13226022.4,51.223155,46.444301,48.285902,0.883305,...,0.581484,0.461950,0.279382,0.294244,0.127132,0.081844,1.29635,1.29635,1.22222,1.20226


In [16]:
#maxnew = np.ones_like(ratios)
#maxnew

In [17]:
#addmin = mintest.tolist()
#addmax = maxtest.tolist()
#data = pd.DataFrame({"max":maxtest,"min":mintest},dtype='float')
#data = data.append(addmin,ignore_index=True)
#data1 = pd.DataFrame(maxtest)
#data2 = pd.DataFrame(mintest)
#data = pd.DataFrame(data1,data2)

In [18]:
#datamul = data.mul([2,1],axis='columns')
#datamul = data['max'].mul(ratios[7])

In [19]:
#datamul

In [20]:
#df = ada2021.loc[(ada2021['max10']>0)& (ada2021['min10']>0)]
df = ada2021
#len(df['date'])
average = df['max5']-df['min5']
avg = average.mean()
avg
df
#point = df.loc[(df['max5'].diff()>0)]
#pointm = df.loc[(df['min5'].diff()>0)]

df['support5']= df.min5.rolling(5,5).min()
df['support10']= df.min10.rolling(10,10).min()
df['resist5']= df.max5.rolling(5,5).max()
df['resist10']= df.max10.rolling(10,10).max()

#df['RSI'] = RSI(df['close'])
#df['STRSI4']=STRSIA(df['close'],56,12,12)
#df['STRSI8']=STRSIA(df['close'],112,24,24)


point = df[(df['max5'] > df['max5'].shift(1)) &  (df['max5'].shift(1) > df['max5'].shift(2))]
pointm = df[(df['min5'] < df['min5'].shift(1)) &  (df['min5'].shift(1) > df['min5'].shift(2))]
point
#ada2021
pointm

for row in range(0,len(pointm)):
    print(pointm['support5'])

19674    1.18150
19681    1.17171
19693    1.19000
19720    1.20432
19736    1.15200
19797    1.19500
19809    1.22250
19819    1.15880
19834    1.17539
19845    1.18790
19852    1.20000
19863    1.20100
19872    1.19134
19883    1.19157
19891    1.20223
19902    1.19900
19937    1.26350
19949    1.29450
19967    1.36933
19978    1.38000
19995    1.37500
20004    1.42190
20011    1.42809
20018    1.44441
20039    1.35000
20050    1.41226
20065    1.35000
Name: support5, dtype: float64
19674    1.18150
19681    1.17171
19693    1.19000
19720    1.20432
19736    1.15200
19797    1.19500
19809    1.22250
19819    1.15880
19834    1.17539
19845    1.18790
19852    1.20000
19863    1.20100
19872    1.19134
19883    1.19157
19891    1.20223
19902    1.19900
19937    1.26350
19949    1.29450
19967    1.36933
19978    1.38000
19995    1.37500
20004    1.42190
20011    1.42809
20018    1.44441
20039    1.35000
20050    1.41226
20065    1.35000
Name: support5, dtype: float64
19674    1.18150
196

In [23]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


#Declare figure


fig = go.Figure()

fig = make_subplots(rows=8, cols=1,subplot_titles=("Price","empty","marge","RSI 1 4 8", "Stoch 1", "Stoch 4","Stoch 8"))
    

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI'],
        name="RSI",
        text="RSI",
        textposition="top center"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI4'],name="RSI4"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI8'],name="RSI8"),
    row=4, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI'],name="STRSI"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK'],name="STRSIK"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID'],name="STRSID"),
    row=5, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI4'],name="STRSI4"),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK4'],name="STRSIK4"),
    row=6, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID4'],name="STRSID4"),
    row=6, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI8'],name="STRSI8"),
    row=7, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK8'],name="STRSIK8"),
    row=7, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID8'],name="STRSIK8"),
    row=7, col=1
)
#add Traces

fig.add_trace(go.Candlestick(x=df.date,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'], name = 'market data'),row=1,col=1)

#fig.add_trace(go.Scatter(x=df['date'], y=df['max10']))

#fig.add_trace(go.Scatter(x=df['date'], y=df['min10']))

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist5']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support5']))


#fig.add_shape(type="line",
#    x0=pointm['date'], y0=pointm['support5'], x1=max(pointm['date']), y1=pointm['support5'],
#    line=dict(
#        color="LightSeaGreen",
#        width=4,
#        dash="dashdot",
#    ))
fig.update_shapes(dict(xref='x', yref='y'))

#for index in range(0,len(pointm)):
    #fig.add_hline(y=pointm['support5'])

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist10']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support10']))
#for i in range(len(df['date'])):
    #fig.add_hline(y=df[i]['min10'], line_dash="dot",
                  #annotation_text="support", 
                  #annotation_position="bottom right")

#fig.add_hline(y=df['max10'], line_dash="dot",
#              annotation_text="resitance", 
#              annotation_position="bottom right")



fig.update_layout(height=3000, width=800, title_text="top sub plots")
# X and Y Axis Update
#fig.update_xaxes(
#    title = 'Date',rangeslider_visible=True
#)

#fig.update_yaxes(
#    title = 'Share Price (USD)'
#)

#Show
fig.show()